# AutoGen मूलभूत नमुना

या कोड नमुन्यात, तुम्ही [AutoGen](https://aka.ms/ai-agents/autogen) AI फ्रेमवर्कचा वापर करून एक मूलभूत एजंट तयार कराल.

या नमुन्याचा उद्देश तुम्हाला त्या पायऱ्या दाखवणे आहे ज्या आपण नंतरच्या कोड नमुन्यांमध्ये वेगवेगळ्या एजंटिक पद्धती अंमलात आणताना वापरणार आहोत.


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## क्लायंट तयार करा

या नमुन्यात, आम्ही [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) वापरून LLM साठी प्रवेश मिळवणार आहोत.

`model` म्हणून `gpt-4o-mini` परिभाषित केले आहे. GitHub Models मार्केटप्लेसमध्ये उपलब्ध असलेल्या दुसऱ्या मॉडेलमध्ये बदल करून वेगवेगळे परिणाम पाहण्याचा प्रयत्न करा.

एक जलद चाचणी म्हणून, आपण एक साधा प्रॉम्प्ट चालवणार आहोत - `What is the capital of France`.


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## एजंटची व्याख्या

आता आपण `client` सेटअप केला आहे आणि तो कार्यरत असल्याची पुष्टी केली आहे, चला `AssistantAgent` तयार करूया. प्रत्येक एजंटला खालील गोष्टी नियुक्त करता येतात:
**name** - एक संक्षिप्त नाव जे मल्टी-एजंट फ्लोमध्ये संदर्भासाठी उपयुक्त ठरेल.
**model_client** - तुम्ही मागील टप्प्यात तयार केलेला क्लायंट.
**tools** - एजंटला एखादे कार्य पूर्ण करण्यासाठी वापरता येणारी उपलब्ध साधने.
**system_message** - LLM चा कार्य, वर्तन आणि टोन परिभाषित करणारा मेटाप्रॉम्प्ट.

तुम्ही system message बदलू शकता आणि पाहू शकता की LLM कसा प्रतिसाद देतो. आपण `tools` बद्दल चौथ्या धड्यात चर्चा करू.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## एजंट चालवा

खालील फंक्शन एजंट चालवेल. आम्ही `on_message` पद्धत वापरतो जेणेकरून नवीन संदेशासह एजंटची स्थिती अपडेट करता येईल.

या प्रकरणात, आम्ही वापरकर्त्याचा नवीन संदेश `"Plan me a great sunny vacation"` सह स्थिती अपडेट करतो.

तुम्ही संदेशाची सामग्री बदलू शकता आणि पाहू शकता की LLM वेगवेगळ्या प्रकारे कसा प्रतिसाद देतो.


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**अस्वीकरण**:  
हा दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) चा वापर करून भाषांतरित करण्यात आला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी, कृपया लक्षात घ्या की स्वयंचलित भाषांतरांमध्ये त्रुटी किंवा अचूकतेचा अभाव असू शकतो. मूळ भाषेतील मूळ दस्तऐवज हा अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर केल्यामुळे उद्भवणाऱ्या कोणत्याही गैरसमज किंवा चुकीच्या अर्थासाठी आम्ही जबाबदार राहणार नाही.
